In [ ]:
# Installing EfficientNet model without internet due to competition requisites

import os
import sys

sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

import torch
os.listdir("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")


from efficientnet_pytorch import EfficientNet


#!pip install efficientnet_pytorch torchtoolbox

In [ ]:
from efficientnet_pytorch import EfficientNet
# Imports here
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
import tifffile as tiff

# For re-importing python modules
import importlib

In [ ]:
# #use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
# These image augmentation functions have already been defined in another notebook.
# Using these functions to pre-process test images before running them through the model.

def compute_statistics(img):
    #img = skimage.io.MultiImage(image)[2]
    width, height = img.shape[0], img.shape[1]
    num_pixels = width * height
    
    num_white_pixels = 0
    green_values_img = 0
    blue_values_img = 0
    
    pixelsum=np.sum(img,axis=-1)
    num_white_pixels = int((pixelsum>620).sum())
    
    green_values_img=np.sum(img[:,:,1])
    blue_values_img=np.sum(img[:,:,2])
    
    ratio_white_pixels = num_white_pixels / num_pixels
    green_concentration = green_values_img / num_pixels
    blue_concentration = blue_values_img / num_pixels
    
    return ratio_white_pixels, green_concentration, blue_concentration


def select_k_best_regions(regions, k=16):
    best_regions = [x for x in regions if x[3] > 0 and x[4] > 0]
    k_best_regions = sorted(best_regions, key=lambda tup: tup[2])[:k]
    
    return k_best_regions


def get_k_best_regions(coordinates, image, window_size=512):
    regions = {}
    for i, tup in enumerate(coordinates):
        x, y = tup[0], tup[1]
        regions[i] = image[x : x+window_size, y : y+window_size, :]
    
    return regions


def generate_patches(image, window_size=200, stride=135, k=20):
    #image = skimage.io.MultiImage(slide_path)[1]
    image = np.array(image)
    
    max_width, max_height = image.shape[0], image.shape[1]
    regions_container = []
    i = 0
    
    while window_size + stride*i <= max_height:
        j = 0
        
        while window_size + stride*j <= max_width:            
            x_top_left_pixel = j * stride
            y_top_left_pixel = i * stride
            
            patch = image[
                x_top_left_pixel : x_top_left_pixel + window_size,
                y_top_left_pixel : y_top_left_pixel + window_size,
                :]
            
            ratio_white_pixels, green_concentration, blue_concentration = compute_statistics(patch)
            
            region_tuple = (x_top_left_pixel, y_top_left_pixel, ratio_white_pixels, green_concentration, blue_concentration)
            regions_container.append(region_tuple)
            
            j += 1
        
        i += 1
    
    k_best_region_coordinates = select_k_best_regions(regions_container, k=k)
    k_best_regions = get_k_best_regions(k_best_region_coordinates, image, window_size)
    #excess_overlap_removed = remove_overlaps(k_best_regions, k_sample=k, k_final=16)
    
           
    return image, k_best_region_coordinates, k_best_regions #, excess_overlap_removed


def display_images(regions, title):
    fig, ax = plt.subplots(6, 6, figsize=(20, 20))
    
    for i, region in regions.items():
        ax[i//6, i%6].imshow(region)
    
    fig.suptitle(title)

    
def glue_to_one_picture(image_patches, window_size=200, k=16):
    side = int(np.sqrt(k))
    image = np.zeros((side*window_size, side*window_size, 3), dtype=np.int16)
        
    for i, patch in image_patches.items():
        x = i // side
        y = i % side
        image[
            x * window_size : (x+1) * window_size,
            y * window_size : (y+1) * window_size,
            :
        ] = patch
    
    return image

In [ ]:
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        #img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(image_id) +".tiff")
        img = skimage.io.MultiImage(img_path)[1]
        
        image, best_coordinates, best_regions = generate_patches(img, window_size=128, stride=128, k=36)
        glued_img = glue_to_one_picture(best_regions, window_size=128, k=36)
        
        image = torch.from_numpy(glued_img).permute(2,0,1).float()
        
        
        
        #Image.MAX_IMAGE_PIXELS = None
                
        #image.transform = transforms.RandomResizedCrop(224)
        
        #y_label = torch.tensor(int(self.annotations.iloc[index,:]['isup_grade']))
        #isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        
        #label = np.zeros(6).astype(np.float32)
        #y_label = label[isup_grade] = 1.
        #y_label = torch.tensor(y_label)
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            #transforms.Resize((int(1840/2),int(1728/2))), 
                                            transforms.ToTensor()])
                                            #transforms.Normalize([0.485, 0.456, 0.406],
                                                             #      [0.229, 0.224, 0.225])])
        if self.transform:
            image = self.transform(image)
        
        return (image, np.array(0))#1840, 1728

In [ ]:
# Loading model

model = EfficientNet.from_name('efficientnet-b0')
model.load_state_dict(torch.load("../input/efficientnet-pytorch/efficientnet-b0-08094119.pth"))
model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 216),
                          nn.ReLU(),
                          nn.Linear(216, 36, bias=True),
                          nn.ReLU(),
                          nn.Linear(36, 5, bias=True))

# Model path
model_path = '../input/arutema-model-36x128x128/Arutema_base_model_w_logits_36x128x128.pth'

# Load checkpoint
checkpoint = torch.load(model_path)

# Load model parameters
model.load_state_dict(checkpoint['model_state_dict'])
model._fc.load_state_dict(checkpoint['classifier_state_dict'])

# Load other model related components
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

In [ ]:
# For submission
submission_test_path = "/kaggle/input/prostate-cancer-grade-assessment/test_images/"
csv_file_dir = '/kaggle/input/prostate-cancer-grade-assessment/test.csv'

pred_y_excel = []
submission=None
if os.path.exists(submission_test_path):
    #print("dir found")
    test_df = pd.read_csv(csv_file_dir)
    test_data = load_csv(csv_file=csv_file_dir, root_dir=submission_test_path)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    #model.eval()
    for ii2, (inputs2, labels2) in enumerate(test_loader):
        inputs2, labels2 = inputs2.to(device), labels2.to(device)
    
        output2 = model.forward(inputs2)
        pred_y2 = output2.sigmoid().sum(1).round().detach()

        pred_y_excel.append(int(pred_y2))
    #print("writing csv")
    #pred_y_excel = [3 for i in range(test_df.shape[0])]
    submission = pd.DataFrame({'image_id':test_df.image_id, 'isup_grade':pred_y_excel})
    submission.to_csv("submission.csv", sep=",", index=False)
else:
    #print("dir not found, building dummy")
    test_df = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv")
#     test_df['isup_grade'] = 0
#     fake_submission=test_df[['image_id','isup_grade']]
    test_df.to_csv('submission.csv', index=False)
    #print('submission saved')
    #print(fake_submission)